# Imports

In [88]:
import os
import pandas as pd
import numpy as np
import math
import cmath
import matplotlib.pyplot as plt
import seaborn as sns

# Data

## Declaration

In [89]:
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 300)

In [90]:
transx = pd.read_csv("../data/transactions.csv")
stores = pd.read_csv("../data/stores.csv")
oil = pd.read_csv("../data/oil.csv")
holidays = pd.read_csv("../data/holidays_events.csv")

training = pd.read_csv("../data/train.csv")
testing = pd.read_csv("../data/test.csv")
sample = pd.read_csv("../data/sample_submission.csv")

## Attributes

### Transactions

In [91]:
transx.head()

display(transx.head())

print(f"Shape: {transx.shape} \n")
print(f"Size: {transx.size} \n")
print(f"Nulls: {transx.isnull().sum()} \n")
print(f"Dtypes: {transx.dtypes} \n")

print(f"Date check: {type(transx['date'][0])}")

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


Shape: (83488, 3) 

Size: 250464 

Nulls: date            0
store_nbr       0
transactions    0
dtype: int64 

Dtypes: date            object
store_nbr        int64
transactions     int64
dtype: object 

Date check: <class 'str'>


Date will need to be converted into a datetime object. Values to be explored. 

### Stores

In [92]:
display(stores.head())

print(f"Shape: {stores.shape} \n")
print(f"Size: {stores.size} \n")
print(f"Nulls: {stores.isnull().sum()} \n")
print(f"Dtypes: {stores.dtypes} \n")

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


Shape: (54, 5) 

Size: 270 

Nulls: store_nbr    0
city         0
state        0
type         0
cluster      0
dtype: int64 

Dtypes: store_nbr     int64
city         object
state        object
type         object
cluster       int64
dtype: object 



Store number could be used here to match up to transactions. There are only 54 rows. Will be checking upon the join to transactions if any stores are omitted.  

### Oil

In [93]:
display(oil.head())

print(f"Shape: {oil.shape} \n")
print(f"Size: {oil.size} \n")
print(f"Nulls: {oil.isnull().sum()} \n")
print(f"Dtypes: {oil.dtypes} \n")

print(type(oil["date"][0]))

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


Shape: (1218, 2) 

Size: 2436 

Nulls: date           0
dcoilwtico    43
dtype: int64 

Dtypes: date           object
dcoilwtico    float64
dtype: object 

<class 'str'>


Date will need to be converted to datetime objects. Need to make sense of the oil price column. There are also nulls. Must address what to do if dates don't line up. It's likely best to observe how much oil prices fluctuate and then extrapolate if possible. 

### Holidays

In [94]:
holidays.head()

display(holidays.head())

print(f"Shape: {holidays.shape} \n")
print(f"Size: {holidays.size} \n")
print(f"Nulls: {holidays.isnull().sum()} \n")
print(f"Dtypes: {holidays.dtypes} \n")

print(f"Date check: {type(holidays['date'][0])}")

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


Shape: (350, 6) 

Size: 2100 

Nulls: date           0
type           0
locale         0
locale_name    0
description    0
transferred    0
dtype: int64 

Dtypes: date           object
type           object
locale         object
locale_name    object
description    object
transferred      bool
dtype: object 

Date check: <class 'str'>


Date needs to be converted to datetime object. Holidays will be useful for understanding any captured anomalies our model may pickup. We will keep them in mind when interpreting inaccuracy in scores or any explanation of our model's limits. These anamolies are ordinal, as well. We can likely fine tune a model to understand these events and treat them in a specific way that best helps us plan for our business' supply and demand. 

### Training

In [95]:
training.head()

display(training.head())

print(f"Shape: {training.shape} \n")
print(f"Size: {training.size} \n")
print(f"Nulls: {training.isnull().sum()} \n")
print(f"Dtypes: {training.dtypes} \n")

print(f"Date check: {type(training['date'][0])}")

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


Shape: (3000888, 6) 

Size: 18005328 

Nulls: id             0
date           0
store_nbr      0
family         0
sales          0
onpromotion    0
dtype: int64 

Dtypes: id               int64
date            object
store_nbr        int64
family          object
sales          float64
onpromotion      int64
dtype: object 

Date check: <class 'str'>


Date conversion needed here. Surprisingly no nulls. Will be joining up against `date`, after all the coincident columns have matching datetime keys.  

### Testing

In [96]:
testing.head()

display(testing.head())
print(f"Shape: {testing.shape} \n")
print(f"Size: {testing.size} \n")
print(f"Nulls: {testing.isnull().sum()} \n")
print(f"Dtypes: {testing.dtypes} \n")

print(f"Date check: {type(testing['date'][0])}")

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


Shape: (28512, 5) 

Size: 142560 

Nulls: id             0
date           0
store_nbr      0
family         0
onpromotion    0
dtype: int64 

Dtypes: id              int64
date           object
store_nbr       int64
family         object
onpromotion     int64
dtype: object 

Date check: <class 'str'>


So testing data omits some of the other data. The question now begs is how do we correctly apply all the data we have to work with to plug into the testing data? 

## Cleaning

### Datetime Conversion

In [97]:
## Datetime conversion function

dated_dfs = [transx, oil, holidays, training]
# testing["date"] = pd.to_datetime(testing["date"]) #this is to show how we'll tag our testing df with all the pre-processing we need {A function will be the result of housing all the cleaning}


for df in dated_dfs:
    df["date"] = pd.to_datetime(df["date"]).dt.date
    print(type(df["date"][0]))

<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>
<class 'datetime.date'>


With the proper conversion of our dataframes' `"date"` column, we have been able to promote their data types into more charismatic data. Pandas is now able to recognize the column with ordinality instead of solely recognizing it as a nominal data type - which it was doing previously. From the perspective of the human eye, we readers understand the column implicitly as something with ordinality. Computers have simple thinking, where the concept of ordinality is not implicitly understood upon first glance - only able to access each object's attribute just a string.

Furthermore, some comments were included in the above cell to reference how we will conduct a similar process on our testing data. The functions housing such will be provided at the top of the notebook.

### Value Exploration

#### Datetime Exploration

In [98]:
##Minimums and Maximums on Training Data alone (we won't be able to understand any testing data)

dated_df_names = ["Transactions:", "Oil:", "Holidays:", "Training:"]

for index, df in enumerate(dated_dfs):
    print(f"Showing table for {dated_df_names[index]}")
    print(f"Min: {df['date'].min()}")
    print(f"Max: {df['date'].max()} \n")

Showing table for Transactions:
Min: 2013-01-01
Max: 2017-08-15 

Showing table for Oil:
Min: 2013-01-01
Max: 2017-08-31 

Showing table for Holidays:
Min: 2012-03-02
Max: 2017-12-26 

Showing table for Training:
Min: 2013-01-01
Max: 2017-08-15 



We notice several years worth of data here (which sounds great!). `holidays` records the largest spread of dates with an absolute minimum and maximum respectively `2012-03-02` and `2017-12-26`. The other tables (`transx`, `oil`, and `training`) all showcase a start date of `2013-01-01`. `oil` extends beyond `transx`'s and `trainings`'s maximums of `2017-08-15` which an end date of `2017-08-31`. The mismatch in size will be noted and addressed as we press forward. 

Additional note: We only focus on the `training` ( + complementary datasets we are able to leverage) data as it would be mostly impractical to perform the same analysis on our `testing` data. We won't be able to understand our unknown data's date range until we already begin sicking our model on the data. This will also prevent any leakage of data and force us to work around real-world constraints. However, the work we can accomplish on the training data might be generalized enough to consider sensible kpis built off of our `date` column, which could be implemented. 

In [ ]:
#date_differences checking

for index, dated_df in enumerate(dated_dfs):
    max_date = dated_df['date'].max() 
    min_date = dated_df['date'].min()

    print(f"Date difference of {dated_df_names[index]}:")
    print(f"{(max_date - min_date).days} days <- ({max_date} - {min_date})  \n")

#TODO: re-approach the issue and its relevance to wards modeling. 
# def date_subtraction(df) #not necessary I don't think for now. 

#date_counts checking

 

Date difference of Transactions::
1687 days <- (2017-08-15 - 2013-01-01)  

Date difference of Oil::
1703 days <- (2017-08-31 - 2013-01-01)  

Date difference of Holidays::
2125 days <- (2017-12-26 - 2012-03-02)  

Date difference of Training::
1687 days <- (2017-08-15 - 2013-01-01)  



We expected this data to have differences in days. So this is unsurprising. However, the days difference between the training and transactions table is effectively the same. What could this mean? 

In [ ]:
#days between per df


In [ ]:
#unique holidays per year

### Null Handling

In [ ]:
def data_cleaning(df):
    # Datetime Conversions
    df["date"] = pd.to_datetime(df["date"])

    # Null Handling
    pass

## Merges

### Pre-merge Analysis

### Merge and Review

In [ ]:
#value count the new dates' instances

#collect the omitted dates from each table from the final table

# Plotting and Feature Exploration

### Sales Aggregations

# Modeling

## Modeling

**Model pre-processing: transformation**

**Modeling**

## Plotting & Analysis

I might make a trackable hyperparameter tuning table. So take all the hyperparameters and showcase the progress of work. 

## Conclusion